In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

In [2]:
#data augmentation and normalisation
stats = ((0.1307 ), (0.3081)) #these will be used as mean and std dev of mnist data

train_transformation = transforms.Compose([transforms.RandomCrop(28,padding=4,padding_mode='reflect'),
                                           transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                           transforms.Normalize(*stats,inplace=True)])

In [3]:
down_train_dataset = datasets.MNIST(root='data/', transform = train_transformation)
train_dl = DataLoader(down_train_dataset,128,shuffle=False,num_workers = 1,pin_memory = True)

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
for x,y in train_dl:
    print(x.shape)
    print(y.shape)
    break

torch.Size([128, 1, 28, 28])
torch.Size([128])


In [5]:
from PIL import Image
import dill
from typing import Iterator

class myMNIST(datasets.MNIST):
    def __init__(self,root,transform):
        super(myMNIST, self).__init__(root, transform=transform)


    def __getitem__(self,index: int):
        self.data, self.targets = datasets.MNIST._load_data(self)
        img  = self.data[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)
        return img

    def __len__(self) -> int:
        self.data, self.targets = datasets.MNIST._load_data(self)
        return len(self.data)


In [6]:
def worker_init_fn():
    worker_info = torch.utils.data.get_worker_info()
    dataset = worker_info.dataset
    worker_id = worker_info.id
    split_size = len(dataset.data)//worker_info.num_workers
    
    dataset.data = dataset.data[worker_id*split_size:(worker_id+1)*split_size]

https://discuss.pytorch.org/t/cant-pickle-local-object-dataloader-init-locals-lambda/31857/26

In [7]:
my_train_dataset = myMNIST(root='data/',transform = train_transformation)

In [8]:
len(my_train_dataset)

60000

In [23]:
dl = DataLoader(my_train_dataset, 128, shuffle = True, num_workers = 0, 
                pin_memory = False, worker_init_fn = worker_init_fn)

In [24]:
for i in dl:
    print(i.shape)
    break

torch.Size([128, 1, 28, 28])


https://medium.com/speechmatics/how-to-build-a-streaming-dataloader-with-pytorch-a66dd891d9dd

In [25]:
# from pathos.multiprocessing import ProcessingPool as pool
# import workers
# if __name__ ==  '__main__': 
#     num_processors = 3
#     p=pool(processes = num_processors)
#     list = [i for i in dl]
#     output = p.map(workers.find_out,list)
#     print(output)

# lets make dataloader and check

In [26]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('gpu')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if ininstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    else:
        data.to(device,non_blocking = True)
    
    
class DeviceDataLoader():
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch,self.device)
            
    def __len__(self):
        return len(self.dl)
        

In [27]:
device = get_default_device()

In [28]:
final_dl = DeviceDataLoader(dl,device)

In [29]:
for i in dl:
    print(i.shape)
    break
    

torch.Size([128, 1, 28, 28])


# no use still ipython cannot do workers processes
